# Test model in notebook

In [ ]:
import sys
sys.path.append("../app")
from main import get_stroke_prob

example_payload = {
    "gender": "Male",
    "age": 67.0,
    "ever_married": "Yes",
    "work_type": "Self-employed",
    "Residence_type": "Urban",
    "avg_glucose_level": 228.69,
    "bmi": 36.6,
    "smoking_status": "smokes",
}
get_stroke_prob(example_payload)

# Test API

In [ ]:
import requests

API_URL = "http://127.0.0.1:5000/predict"

response = requests.post(API_URL, json=example_payload)

print("Status code:", response.status_code)
print("Response JSON:", response.json())